# Generate data for all figures and export as CSV files for plotting in TikZ

### Subfunction to generate the grids to evaluate the models over

In [1]:
function create_grids(ca, V)
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur = Array(Any, (1*length(V),))
    x_grid_cur_dense = Array(Any, (1*length(Vdense),))
    iterind = 0;
    for j1 in product([ca], V)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    iterind = 0;
    for j1 in product([ca], Vdense)
        iterind += 1;
        x_grid_cur_dense[iterind] = collect(j1)
    end
        
    return x_grid_cur, x_grid_cur_dense
end

create_grids (generic function with 1 method)

# Figure 1

In [2]:
# Figure 1 A - Drawn in TikZ

In [3]:
# Figure 1 B and C

# Load the BK functions, the x_grid(Ca, V) that we evaluate over and the initial condition phi0
include("../BK_functions/bk_setup_script.jl"); 

Vdense = minimum(V):3e-3:maximum(V) # voltage sampled every 3 millivolts for smooth lines

for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for original model
    tmp = BK_simulator(phi0, hcat(x_grid_cur...), model_id=10)
    writecsv("CSV/figure_1b_panel1_ca$(ca_str)um.csv", hcat(V[:]*1000, tmp[:]))
    
    # Save also the log10 values
    writecsv("CSV/figure_1c_panel1_ca$(ca_str)um.csv", hcat(V[:]*1000, log10(tmp[:])))
    
    # Densely sampled
    tmp = BK_simulator(phi0, hcat(x_grid_cur_dense...), model_id=10)
    writecsv("CSV/figure_1b_panel2_ca$(ca_str)um.csv", hcat(Vdense[:]*1000, tmp[:]))
    writecsv("CSV/figure_1c_panel2_ca$(ca_str)um.csv", hcat(Vdense[:]*1000, log10(tmp[:])))
end

# Figure 2

In [4]:
# Figure 2 A
tmp = load("results/noisy_fits_8.jld")["noisy_fits"][10] # Example noisy fit with diverging parameters
phi1 = tmp["phi1_orig"] # Get the fitted parameter vector

# Output the simulated data point markers in a CSV format with: 
# first column - voltage (mV), second column - Popen, third column error top, fourth column

# Also output the example fit with just first col - voltage (mV), second col - Popen
# but with voltage sampled more densely to make it a continous line

for ca in Ca  
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for original model
    tmp = BK_simulator(phi0, hcat(x_grid_cur...), model_id=0)
    errbar = tmp.*0.1 # Generate the error bar for every data point.
    writecsv("CSV/figure_2a_panel1_ca$(ca_str)um.csv", hcat(V[:].*1000, tmp[:], errbar[:], errbar[:]))
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(phi1, hcat(x_grid_cur_dense...), model_id=0)
    writecsv("CSV/figure_2a_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

In [5]:
# Figure 2 B
# Output the original and the fitted parameter values (first and second column of CSV respectively)
writecsv("CSV/figure_2b_panel1.csv", [phi0 phi1])

# Figure 3

In [6]:
# Figure 3 A
# Table with theoretical lower bound on parameter uncertainties

# Calculation based on
# Gutenkunst et al, 2007, PLoS Comp Bio, Universally Sloppy Parameter Sensitivities in Systems Biology Models

# Load the MBAM outputs
using JLD
outp = load("results/MBAM_outp_orig.jld", "outp");
outp_log10 = load("results/MBAM_outp_log10.jld", "outp_log10");

# Recreate the (approximate) Hessians based on the eigenvalue decomposition
invHess_orig = outp[4][1,1] * diagm(1./outp[3][1,1]) * inv(outp[4][1,1])
invHess_log10 = outp_log10[4][1,1] * diagm(1./outp_log10[3][1,1]) * inv(outp_log10[4][1,1])

noise_level = 0.1; # 10% noise assumed

# Individual parameter uncertanties:
uncert_orig = exp(4*sqrt((noise_level./size(hcat(x_grid...))[2]).*diag(invHess_orig))) - 1;
uncert_log10 = exp(4*sqrt((noise_level./size(hcat(x_grid...))[2]).*diag(invHess_log10))) - 1;

In [7]:
# Save the numbers to CSV, multiply by 100 to get percentage values used in the table
writecsv("CSV/figure_3a_panel1.csv", [uncert_orig*100 uncert_log10*100])
[uncert_orig*100 uncert_log10*100]

8x2 Array{Float64,2}:
     5.44031e11  7917.07   
 40713.2          612.175  
  7463.71         262.106  
    71.8033         9.50224
   304.318         13.1469 
  2869.05          32.8843 
     1.88338e6   1200.63   
 17874.6           22.1876 

In [8]:
# Figure 3 B - Drawn in TikZ

In [9]:
# Figure 3 C
using JLD
outp = load("results/MBAM_outp_orig.jld", "outp");
outp_log10 = load("results/MBAM_outp_log10.jld", "outp_log10");

writecsv("CSV/figure_3c_panel1_orig.csv", hcat(1:size(outp[3][1,1],1), log(outp[3][1,1])))
writecsv("CSV/figure_3c_panel1_log10.csv", hcat(1:size(outp_log10[3][1,1],1), log(outp_log10[3][1,1])))

# Figure 4

In [10]:
# Figure 4 A
# Cost for original space model
# writecsv("CSV/figure_4a_panel1.csv", hcat(0:size(outp[1],1), vcat(outp[2][1,1], outp[2][:,2])))
writecsv("CSV/figure_4a_panel1.csv", hcat(0:5, vcat(outp[2][1,1], outp[2][1:5,2])))

In [11]:
# Figure 4 B
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 B
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp[1][4,1]), hcat(x_grid_cur_dense...), model_id=13)
    writecsv("CSV/figure_4b_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

In [12]:
# Figure 4 C
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 B
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp[1][5,1]), hcat(x_grid_cur_dense...), model_id=14)
    writecsv("CSV/figure_4c_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

# Figure 5 / Figure A1

Show the MBAM reduction steps (in Figure 5 schematically, in Figure A1 with more details)

* Panel n - nth reduction containing each parameter curve

In [13]:
# Naming the original and reduced parameter vectors for each model reduction step

param_names = Array(Array{AbstractString},size(outp[1],1));
param_names[1] = ["L0", "zL", "J0", "zJ", "Kd", "C", "D", "E"]
param_names[2] = ["L0", "J0", "zJ", "Kd", "C", "D", "E"]
param_names[3] = ["J0", "zJ", "Kd", "LC","LD", "EL"];
param_names[4] = ["J0", "zJ", "Kd", "CE","LD"];
param_names[5] = ["LDJ", "zJ", "Kd", "CE"];
param_names[6] = ["LDJ", "zJ", "CEKd"];

In [14]:
for model_red = 1:size(outp[1],1)
    # Get the log param values during the reduction
    phi_red_time = outp[5][model_red]
    phi_red_vals = hcat(outp[6][model_red]...)
    phi_red_vals = phi_red_vals[1:div(size(phi_red_vals,1),2),:]
    
    # Find the converged time (the one after no parameters change)
    cutoff = 0
    for i1 = 2:size(phi_red_vals,2)
        if all(phi_red_vals[:,i1].==phi_red_vals[:,i1-1])
            cutoff = i1
            break
        end
    end
    
    # Produce the geodesic time - log(value) for each parameter for each reduction step
    for param_num in 1:size(phi_red_vals,1)
        writecsv("CSV/figure_5"*string("abcdef"[model_red])*"_panel1_param_$(param_names[model_red][param_num]).csv", 
            hcat(phi_red_time[1:cutoff], phi_red_vals[param_num,1:cutoff][:]))
    end
end

# Figure 6

In [15]:
# Figure 6 A
# Cost for log space model
writecsv("CSV/figure_6a_panel1.csv", hcat(0:4, vcat(outp_log10[2][1,1], outp_log10[2][1:4,2])))

In [16]:
# Figure 6 B
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 C
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp_log10[1][2,1]), hcat(x_grid_cur_dense...), model_id=52)
    writecsv("CSV/figure_6b_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

In [17]:
# Figure 6 C
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 C
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp_log10[1][3,1]), hcat(x_grid_cur_dense...), model_id=53)
    writecsv("CSV/figure_6c_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

# Figure 7

In [18]:
# Figure 3 A
# Table with theoretical lower bound on parameter uncertainties

# Calculation based on
# Gutenkunst et al, 2007, PLoS Comp Bio, Universally Sloppy Parameter Sensitivities in Systems Biology Models

# Load the MBAM outputs
using JLD
outp = load("saves/figure3_smallVnorm.jld", "outp");
outp_log10 = load("saves/figure4.jld", "outp_log10");


noise_level = 0.1; # 10% noise assumed

for red_num = 1:3

    # Recreate the (approximate) Hessians of the reduction step based on the eigenvalue decomposition
    invHess_orig = outp[4][red_num+1,1] * diagm(1./outp[3][red_num+1,1]) * inv(outp[4][red_num+1,1])
    invHess_log10 = outp_log10[4][red_num+1,1] * diagm(1./outp_log10[3][red_num+1,1]) * inv(outp_log10[4][red_num+1,1])
    
    # Individual parameter uncertanties:
    uncert_orig = exp(4*sqrt((noise_level./size(hcat(x_grid...))[2]).*diag(invHess_orig))) - 1;
    uncert_log10 = exp(4*sqrt((noise_level./size(hcat(x_grid...))[2]).*diag(invHess_log10))) - 1;
    
    
    # Save the numbers to CSV, multiply by 100 to get percentage values used in the table
    writecsv("CSV/figure_7"*string("abcdef"[red_num])*"_panel1.csv", uncert_orig[:]*100)
    writecsv("CSV/figure_7"*string("abcdef"[red_num])*"_panel2.csv", uncert_log10[:]*100)
    @show uncert_orig*100
    @show uncert_log10*100
end

uncert_orig * 100 = [1.5382982109523043e22,92.66987079018416,59.829892653808955,128.43557770860318,1.8265613845420252e6,1.853866132159499e7,7.576758461584554e6]
uncert_log10 * 100 = [67.82074750170706,43.10685223245243,5.612237333129677,10.719317913788817,16.044257870361722,51.93684615145933,23.076112765139566]
uncert_orig * 100 = [82.34652297993739,36.14735704206695,51.03389726490182,1.2270147337677004e9,96.85890596718811,1.311784163578657e9]
uncert_log10 * 100 = [125.45509395815215,27.473744934547682,3.3805004349855228,10.076296865890487,25.34163109541341,31.791175645710656]
uncert_orig * 100 = [84.52578674529008,17.997631777822765,47.88764260462051,44.786010397309894,68.67389010234042]
uncert_log10 * 100 = [2.0482537376088183,1.6644229932111676,9.664723604268666,56.93274335438731,62.5472917649224]


# Figure 8

In [44]:
include("../BK_functions/bk_setup_script.jl")

# Original and Simulation parameters (fit the orig assay, see Figure 2)
phi1_orig = load("results/noisy_fits_divergent_model_1.jld", "metadata")["phi1_orig"]
[phi0 phi1_orig]

# Load log fits to noisy data simulated with parameters above (both original and diverging)
using JLD
all_vars = Dict();
orig_vars = Dict();
keys = 83
var = "phi1_log"
orig_var = "phi1_log"
n=0
for i in 1:100
    n+=1;
    tmp_div = load("results/noisy_fits_divergent_model_"*string(i)*".jld", "noisy_fits")
    tmp_orig = load("results/noisy_fits_"*string(n)*".jld", "noisy_fits")
    for key in keys
        if !haskey(all_vars, key)
            all_vars[key] = Array(Any, 100);
        end
        if !haskey(orig_vars, key)
            orig_vars[key] = Array(Any, 100);
        end
        all_vars[key][n] = tmp_div[key][var]
        orig_vars[key][n] = tmp_orig[key-31][var]
    end
end

### Output the individual parameter fit results for the base set (from model 52=82) and the diverging set (from model 83)

In [45]:
# Matrix of all fits for J0, zJ, Kd, L0*C, L0*D, E/L0  (base parameters, 1x reduced log fit)
base_fits = vcat(log10(hcat(orig_vars[keys][1:n]...)[[2,3,4],:]),
                log10(hcat(orig_vars[keys][1:n]...)[[5,6],:]) .+ repmat(log10(hcat(orig_vars[keys][1:n]...)[1,:])./4,2,1),
                log10(hcat(orig_vars[keys][1:n]...)[7,:]) .- log10(hcat(orig_vars[keys][1:n]...)[1,:])./4)

# Matrix of all fits for J0, zJ, Kd, L0*C, L0*D, E/L0  (diverging parameters, 2x reduced log fit)
diverg_fits = log10(hcat(all_vars[keys][1:n]...))

# True base params and diverging params for J0, zJ, Kd, L0*C, L0*D, E/L0 
true_params = vcat(log10(hcat(phi0, phi1_orig)[[3,4,5],:]),
log10(hcat(phi0,phi1_orig)[[6,7],:]) .+ repmat(log10(hcat(phi0,phi1_orig)[1,:])./4,2,1),
log10(hcat(phi0,phi1_orig)[8,:]) .- log10(hcat(phi0,phi1_orig)[1,:])./4)

6x2 Array{Float64,2}:
 -0.988853   -1.17861 
 -0.236572   -0.231481
 -4.40894    -4.56175 
 -0.624814   -0.79247 
  0.0684793   0.351451
  1.71542     1.68592 

In [46]:
cloud_membership = ones(n,1).>0; #(diverg_fits[1,:].>mean(diverg_fits[1,:]));

In [47]:
# Export the appropriate CSVs

avg_func(x) = mean(x)


for param_num = 1:6

# True base param vs true diverging param
writecsv("CSV/figure_8"*string("abcdef"[param_num])*"_panel1.csv", hcat([0; 1], true_params[param_num,:][:]))
# Average fits
writecsv("CSV/figure_8"*string("abcdef"[param_num])*"_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[param_num,:]), avg_func(diverg_fits[param_num,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8"*string("abcdef"[param_num])*"_panel3.csv", hcat(randn(n,1)*0.1, base_fits[param_num,:]'))
writecsv("CSV/figure_8"*string("abcdef"[param_num])*"_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[param_num, cloud_membership]'))
writecsv("CSV/figure_8"*string("abcdef"[param_num])*"_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[param_num, !cloud_membership]'))

end

### Compensation between C and E

In [11]:
# Matrix of all fits for C and E (base parameters, reduced log fit)
#base_fits = log10(hcat(orig_vars[keys][1:n]...)[[5,7],:])

# Matrix of all fits for C and E (diverging parameters, reduced log fit)
#diverg_fits = log10(hcat(all_vars[keys][1:n]...)[[5,7],:])

# True base params and diverging params for C and E
#true_params = log10(hcat(phi0[[6,8]], phi1_orig[[6,8]]))



In [12]:
# Matrix of all fits for C and E (base parameters, reduced log fit)
base_fits = log10(hcat(orig_vars[keys][1:n]...)[[5,7],:])

# Matrix of all fits for C and E (diverging parameters, reduced log fit)
diverg_fits = log10(hcat(all_vars[keys][1:n]...)[[4,6],:])

# True base params and diverging params for C and E
true_params = log10(hcat(phi0[[6,8]], phi1_orig[[6,8]]))



2x2 Array{Float64,2}:
 0.789581   8.236  
 0.30103   -7.34254

In [13]:
# Defining a coloring split for "diverging all fits" based on whether the fitted 
# C is above or below the true diverging parameter

cloud_membership = (diverg_fits[1,:].>mean(diverg_fits[1,:]));

In [14]:
# Export the appropriate CSVs

avg_func(x) = mean(x)

# Figure A, changes in log10(C)
# True base param vs true diverging param
writecsv("CSV/figure_8a_panel1.csv", hcat([0; 1], true_params[1,:][:]))
# Average fits
writecsv("CSV/figure_8a_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[1,:]), avg_func(diverg_fits[1,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8a_panel3.csv", hcat(randn(n,1)*0.1, base_fits[1,:]'))
writecsv("CSV/figure_8a_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[1, cloud_membership]'))
writecsv("CSV/figure_8a_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[1, !cloud_membership]'))


# Figure B, changes in log10(E)
# True base param vs true diverging param
writecsv("CSV/figure_8b_panel1.csv", hcat([0; 1], true_params[2,:][:]))
# Average fits
writecsv("CSV/figure_8b_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[2,:]), avg_func(diverg_fits[2,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8b_panel3.csv", hcat(randn(n,1)*0.1, base_fits[2,:]'))
writecsv("CSV/figure_8b_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[2, cloud_membership]'))
writecsv("CSV/figure_8b_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[2, !cloud_membership]'))


# Figure C, changes in log10(C*E)
# True base param vs true diverging param
writecsv("CSV/figure_8c_panel1.csv", hcat([0; 1], sum(true_params,1)[:]))
# Average fits
writecsv("CSV/figure_8c_panel2.csv", hcat([0; 1], vcat(avg_func(sum(base_fits,1)), avg_func(sum(diverg_fits,1)))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8c_panel3.csv", hcat(randn(n,1)*0.1, sum(base_fits,1)'))
writecsv("CSV/figure_8c_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, sum(diverg_fits[:, cloud_membership],1)'))
writecsv("CSV/figure_8c_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, sum(diverg_fits[:, !cloud_membership],1)'))

### Compensation between L0 and D

In [15]:
# Matrix of all fits for L0 and D (base parameters, reduced log fit)
# base_fits = log10(hcat(orig_vars[keys][1:n]...)[[1,6],:])

# Matrix of all fits for L0 and D (diverging parameters, reduced log fit)
# diverg_fits = log10(hcat(all_vars[keys][1:n]...)[[1,6],:])

# True base params and diverging params for L0 and D
# true_params = log10(hcat(phi0[[1,7]], phi1_orig[[1,7]]))

In [16]:
# Matrix of all fits for L0 and D (base parameters, reduced log fit)
base_fits = log10(hcat(orig_vars[keys][1:n]...)[[1,6],:])

# Matrix of all fits for L0 and D (diverging parameters, reduced log fit)
diverg_fits = log10(hcat(all_vars[keys][1:n]...)[[5,5],:])

# True base params and diverging params for L0 and D
true_params = log10(hcat(phi0[[1,7]], phi1_orig[[1,7]]))

2x2 Array{Float64,2}:
 -5.65758  -36.1139 
  1.48287    9.37992

In [17]:
# Export the appropriate CSVs

# Figure D, changes in log10(L0)
# True base param vs true diverging param
writecsv("CSV/figure_8d_panel1.csv", hcat([0; 1], true_params[1,:][:]))
# Average fits
writecsv("CSV/figure_8d_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[1,:]), avg_func(diverg_fits[1,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8d_panel3.csv", hcat(randn(n,1)*0.1, base_fits[1,:]'))
writecsv("CSV/figure_8d_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[1, cloud_membership]'))
writecsv("CSV/figure_8d_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[1, !cloud_membership]'))



# Figure E, changes in log10(D)
# True base param vs true diverging param
writecsv("CSV/figure_8e_panel1.csv", hcat([0; 1], true_params[2,:][:]))
# Average fits
writecsv("CSV/figure_8e_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[2,:]), avg_func(diverg_fits[2,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8e_panel3.csv", hcat(randn(n,1)*0.1, base_fits[2,:]'))
writecsv("CSV/figure_8e_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[2, cloud_membership]'))
writecsv("CSV/figure_8e_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[2, !cloud_membership]'))


# Figure F, changes in log10(L0^-0.25 * D)
# True base param vs true diverging param
writecsv("CSV/figure_8f_panel1.csv", hcat([0; 1], sum(true_params.*[0.25,1],1)[:]))
# Average fits
writecsv("CSV/figure_8f_panel2.csv", hcat([0; 1], vcat(avg_func(sum(base_fits.*[0.25,1],1)), avg_func(sum(diverg_fits.*[0.25,1],1)))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8f_panel3.csv", hcat(randn(n,1)*0.1, sum(base_fits.*[0.25,1],1)'))
writecsv("CSV/figure_8f_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, sum(diverg_fits[:, cloud_membership].*[0.25,1],1)'))
writecsv("CSV/figure_8f_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, sum(diverg_fits[:, !cloud_membership].*[0.25,1],1)'))


### Non-compensating parameters - J0, zJ, Kd

In [18]:
# Matrix of all fits for J0, zJ, Kd (base parameters, reduced log fit)
#base_fits = log10(hcat(orig_vars[keys][1:n]...)[[2,3,4],:])

# Matrix of all fits for J0, zJ, Kd (diverging parameters, reduced log fit)
#diverg_fits = log10(hcat(all_vars[keys][1:n]...)[[2,3,4],:])

# True base params and diverging params for J0, zJ, Kd
#true_params = log10(hcat(phi0[[3,4,5]], phi1_orig[[3,4,5]]))

In [19]:
# Matrix of all fits for J0, zJ, Kd (base parameters, reduced log fit)
base_fits = log10(hcat(orig_vars[keys][1:n]...)[[1,2,3],:])

# Matrix of all fits for J0, zJ, Kd (diverging parameters, reduced log fit)
diverg_fits = log10(hcat(all_vars[keys][1:n]...)[[2,3,4],:])

# True base params and diverging params for J0, zJ, Kd
true_params = log10(hcat(phi0[[3,4,5]], phi1_orig[[3,4,5]]))

3x2 Array{Float64,2}:
 -0.988853  -1.17861 
 -0.236572  -0.231481
 -4.40894   -4.56175 

In [20]:
# Export the appropriate CSVs

# Figure G, changes in log10(J0)
# True base param vs true diverging param
writecsv("CSV/figure_8g_panel1.csv", hcat([0; 1], true_params[1,:][:]))
# Average fits
writecsv("CSV/figure_8g_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[1,:]), avg_func(diverg_fits[1,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8g_panel3.csv", hcat(randn(n,1)*0.1, base_fits[1,:]'))
writecsv("CSV/figure_8g_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[1, cloud_membership]'))
writecsv("CSV/figure_8g_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[1, !cloud_membership]'))



# Figure H, changes in log10(zJ)
# True base param vs true diverging param
writecsv("CSV/figure_8h_panel1.csv", hcat([0; 1], true_params[2,:][:]))
# Average fits
writecsv("CSV/figure_8h_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[2,:]), avg_func(diverg_fits[2,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8h_panel3.csv", hcat(randn(n,1)*0.1, base_fits[2,:]'))
writecsv("CSV/figure_8h_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[2, cloud_membership]'))
writecsv("CSV/figure_8h_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[2, !cloud_membership]'))

# Figure I, changes in log10(Kd)
# True base param vs true diverging param
writecsv("CSV/figure_8i_panel1.csv", hcat([0; 1], true_params[3,:][:]))
# Average fits
writecsv("CSV/figure_8i_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[3,:]), avg_func(diverg_fits[3,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8i_panel3.csv", hcat(randn(n,1)*0.1, base_fits[3,:]'))
writecsv("CSV/figure_8i_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[3, cloud_membership]'))
writecsv("CSV/figure_8i_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[3, !cloud_membership]'))
